In [15]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [16]:
np.random.seed(42)

train_num = 50000
test_num = 5000
length = 300
batch_size = 64
epochs = 3
lr = 0.001

t = np.arange(0, length) * (2*np.pi)/length

train_X = np.zeros(shape=(train_num, length))
train_Y = np.zeros(shape=(train_num, length))

test_X = np.zeros(shape=(test_num, length))
test_Y = np.zeros(shape=(test_num, length))

for i in range(train_num + test_num):
    amp = np.random.rand(2)
    vel = 3*np.random.randn(2)
    offset_x = np.random.randn(2)
    offset_y = np.random.randn(2)
    noise = np.random.randn(length) * 0.2
    y = amp[0] * np.sin(vel[0]*t + offset_x[0]) + offset_y[0]
    y += amp[1] * np.cos(vel[1]*t + offset_x[1]) + offset_y[1]
    x = y + noise

    if i < test_num:
        test_X[i] = x
        test_Y[i] = y
    else:
        train_X[i-test_num] = x
        train_Y[i-test_num] = y


In [17]:
naive_model = keras.Sequential()

naive_model.add(keras.layers.Dense(64, input_shape=(length, 1), activation='leaky_relu'))

naive_model.add(keras.layers.LSTM(128, return_sequences=True))

naive_model.add(keras.layers.Dense(1))

naive_model.compile(
    loss = keras.losses.mean_absolute_error,
    optimizer=keras.optimizers.Adam(learning_rate=lr),
    metrics=keras.metrics.mae,
)
naive_model.fit(
    train_X, train_Y, validation_data=(test_X, test_Y), batch_size=batch_size, epochs=epochs, verbose=0
)

naive_predict_Y = naive_model.predict(test_X)
naive_score = naive_model.evaluate(test_X, test_Y)

157/157 [==============================] - 1s 7ms/step - loss: 0.0675 - mean_absolute_error: 0.0675


In [ ]:
# =======================================
# Assignment One
# =======================================

bid_model = keras.Sequential()

bid_model.add(keras.layers.Dense(128, input_shape=(length, 1), activation='leaky_relu'))

bid_model.add(keras.layers.Bidirectional(keras.layers.LSTM(40, return_sequences=True)))

bid_model.add(keras.layers.Bidirectional(keras.layers.LSTM(40, return_sequences=True)))

bid_model.add(keras.layers.Bidirectional(keras.layers.LSTM(40, return_sequences=True)))

bid_model.add(keras.layers.Dense(1))

bid_model.summary()

bid_model.compile(
    loss=keras.losses.mean_absolute_error,
    optimizer=keras.optimizers.Adam(learning_rate=lr),
    metrics=keras.metrics.mae,
)
bid_model.fit(
    train_X, train_Y, validation_data=(test_X, test_Y), batch_size=batch_size, epochs=epochs, verbose=0
)

bid_predict_Y = bid_model.predict(test_X)
bid_score = bid_model.evaluate(test_X, test_Y)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 300, 128)          256       
                                                                 
 bidirectional_6 (Bidirecti  (None, 300, 80)           54080     
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 300, 80)           38720     
 onal)                                                           
                                                                 
 bidirectional_8 (Bidirecti  (None, 300, 80)           38720     
 onal)                                                           
                                                                 
 dense_11 (Dense)            (None, 300, 1)            81        
                                                      

In [ ]:
# =======================================
# Assignment Two
# =======================================

plt.figure(figsize=(12, 4))
vis_idx = 0
plt.plot(test_X[vis_idx], 'y')
plt.plot(naive_predict_Y[vis_idx], 'k')
plt.plot(bid_predict_Y[vis_idx], 'm')
plt.legend(['input', 'naive_pred', '3GRUs_pred'])